In [1]:
import pandas as pd
import numpy as np
import pickle
import time

In [2]:
# heartrate_df = pd.read_csv('data/heart_rate_daily.csv')
hr_df = pd.read_csv('data/heart_rate_daily_new.csv')

In [3]:
dailies_sammaries_df = pd.read_csv(
    'data/dailies_summary.csv', index_col='summaryId')

In [13]:
# some summaryIds from heart_rate_daily.csv are not in dailies_summary.csv
valid_hr_df = heartrate_df[heartrate_df['dailiessummaryId'].isin(
    dailies_sammaries_df.index)]
valid_hr_df

,userId,dailiessummaryId,timeOffsetHeartRateSamples,pulse
0,00a7a796-c572-44d7-a950-7f6bca4a4394,x48cc04e-6313c050-15180-6,33615,83
1,00a7a796-c572-44d7-a950-7f6bca4a4394,x48cc04e-6313c050-15180-6,33630,83
2,00a7a796-c572-44d7-a950-7f6bca4a4394,x48cc04e-6313c050-15180-6,33645,83
3,00a7a796-c572-44d7-a950-7f6bca4a4394,x48cc04e-6313c050-15180-6,33660,83
4,00a7a796-c572-44d7-a950-7f6bca4a4394,x48cc04e-6313c050-15180-6,33675,83
...,...,...,...,...
55575881,test,x47e8f6d-62ab99d0-a8c0-6,42900,64
55575882,test,x47e8f6d-62ab99d0-a8c0-6,43095,70
55575883,test,x47e8f6d-62ab99d0-a8c0-6,43110,70
55575884,test,x47e8f6d-62ab99d0-a8c0-6,43125,70


In [15]:
# split the data into 4 parts because it's too big
hr_df_1 = valid_hr_df.iloc[:len(valid_hr_df)//4]
hr_df_2 = valid_hr_df.iloc[len(valid_hr_df)//4:len(valid_hr_df)//2]
hr_df_3 = valid_hr_df.iloc[len(valid_hr_df)//2:3*len(valid_hr_df)//4]
hr_df_4 = valid_hr_df.iloc[3*len(valid_hr_df)//4:]

In [46]:
# add a column 'globalTime' which is the start time (taken from dailies_summary) + timeOffsetHeartRateSamples
hr_df_1.loc[:, 'globalTime'] = dailies_sammaries_df.loc[hr_df_1['dailiessummaryId'], 'startTimeInSeconds'].to_numpy() \
    + hr_df_1.loc[:, 'timeOffsetHeartRateSamples'].to_numpy()
hr_df_2.loc[:, 'globalTime'] = dailies_sammaries_df.loc[hr_df_2['dailiessummaryId'], 'startTimeInSeconds'].to_numpy() \
    + hr_df_2.loc[:, 'timeOffsetHeartRateSamples'].to_numpy()
hr_df_3.loc[:, 'globalTime'] = dailies_sammaries_df.loc[hr_df_3['dailiessummaryId'], 'startTimeInSeconds'].to_numpy() \
    + hr_df_3.loc[:, 'timeOffsetHeartRateSamples'].to_numpy()
hr_df_4.loc[:, 'globalTime'] = dailies_sammaries_df.loc[hr_df_4['dailiessummaryId'], 'startTimeInSeconds'].to_numpy() \
    + hr_df_4.loc[:, 'timeOffsetHeartRateSamples'].to_numpy()

C:\Users\tomda\AppData\Local\Temp\ipykernel_7812\1330231264.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hr_df_1.loc[:, 'globalTime'] = dailies_sammaries_df.loc[hr_df_1['dailiessummaryId'], 'startTimeInSeconds'].to_numpy() \
C:\Users\tomda\AppData\Local\Temp\ipykernel_7812\1330231264.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hr_df_2.loc[:, 'globalTime'] = dailies_sammaries_df.loc[hr_df_2['dailiessummaryId'], 'startTimeInSeconds'].to_numpy() \
C:\Users\tomda\AppData\Local\Temp\ipykernel_7812

In [50]:
hr_df = pd.concat([hr_df_1, hr_df_2, hr_df_3, hr_df_4])
hr_df

,userId,dailiessummaryId,timeOffsetHeartRateSamples,pulse,globalTime
0,00a7a796-c572-44d7-a950-7f6bca4a4394,x48cc04e-6313c050-15180-6,33615,83,1662272415
1,00a7a796-c572-44d7-a950-7f6bca4a4394,x48cc04e-6313c050-15180-6,33630,83,1662272430
2,00a7a796-c572-44d7-a950-7f6bca4a4394,x48cc04e-6313c050-15180-6,33645,83,1662272445
3,00a7a796-c572-44d7-a950-7f6bca4a4394,x48cc04e-6313c050-15180-6,33660,83,1662272460
4,00a7a796-c572-44d7-a950-7f6bca4a4394,x48cc04e-6313c050-15180-6,33675,83,1662272475
...,...,...,...,...,...
55575881,test,x47e8f6d-62ab99d0-a8c0-6,42900,64,1655456100
55575882,test,x47e8f6d-62ab99d0-a8c0-6,43095,70,1655456295
55575883,test,x47e8f6d-62ab99d0-a8c0-6,43110,70,1655456310
55575884,test,x47e8f6d-62ab99d0-a8c0-6,43125,70,1655456325


In [53]:
# make sure we didn't modified the original dataframe
hr_df.drop('globalTime', axis=1).equals(valid_hr_df)

True

In [54]:
# save the new dataframe
hr_df.to_csv('data/heart_rate_daily_new.csv', index=False)

In [85]:
list(data_0.keys())

['light_sleep',
 'awake_sleep',
 'deep_sleep',
 'epoch',
 'heart_rate',
 'epoch_unique_days',
 'epoch_avg_datapoints__per_day',
 'deep_sleep_unique_days',
 'deep_sleep_avg_datapoints_per_day',
 'awake_sleep_unique_days',
 'awake_sleep_avg_datapoints_per_day',
 'light_sleep_unique_days',
 'light_sleep_avg_datapoints_per_day',
 'sleep_by_minutes',
 'sleep_by_weeks']

In [76]:
light_sleep_intervals = [(start, end) for start, end in zip(
    light_sleep_data['startTimeInSeconds'], light_sleep_data['endTimeInSeconds'])]
light_sleep_intervals
hr_vals = []
for i in range(len(user_hr_data)):
    for start, end in intervals:
        if user_hr_data.loc[i, 'globalTime'] >= start and user_hr_data.loc[i, 'globalTime'] <= end:
            hr_vals.append(user_hr_data.loc[i, 'pulse'])

In [78]:
light_hr_vals = np.array(hr_vals)

50.879122776722504

In [79]:
deep_sleep_data = sleeping[id]['deep_sleep']
deep_sleep_intervals = [(start, end) for start, end in zip(
    deep_sleep_data['startTimeInSeconds'], deep_sleep_data['endTimeInSeconds'])]
hr_vals = []
for i in range(len(user_hr_data)):
    for start, end in deep_sleep_intervals:
        if user_hr_data.loc[i, 'globalTime'] >= start and user_hr_data.loc[i, 'globalTime'] <= end:
            hr_vals.append(user_hr_data.loc[i, 'pulse'])
deep_hr_vals = np.array(hr_vals)
deep_hr_vals.mean()

In [ ]:
with open('data_dict.pickle', 'rb') as f:
    data_dict = pickle.load(f)

In [102]:
# adding rem_sleep to sleeping data
for id in sleeping.keys():
    filtered_rem_sleep = data_dict['rem_sleep'][data_dict['rem_sleep']['userId'] == id] \
        .reset_index().drop(columns=['index', 'summaryId'])
    sleeping[id]['rem_sleep'] = filtered_rem_sleep

In [108]:
# saving the new sleeping data
with open('sleeping.pickle', 'wb') as f:
    pickle.dump(sleeping, f)

In [4]:
with open('sleeping.pickle', 'rb') as f:
    sleeping = pickle.load(f)

In [5]:
sleeping_keys = list(sleeping.keys())

In [46]:
from tqdm import tqdm
from collections import defaultdict

In [47]:
# getting the average heart rate for each user during sleep for light, deep and rem sleep.
# not including 'awake sleep' for now because the average heart rate is much higher.
total_sleep_hr_avg = {}
sleep_types = ('light_sleep', 'deep_sleep', 'rem_sleep')
for id in tqdm(sleeping_keys):
    times = np.vstack(
        [sleeping[id][sleep_type][['startTimeInSeconds', 'endTimeInSeconds']].values for sleep_type in sleep_types]
    )
    if times.size == 0:
        continue
    user_hr_data = hr_df[hr_df['userId'] == id]
    total_hr_count = total_sample_count = 0

    # Create a boolean mask for selecting relevant rows
    mask = np.logical_or.reduce([
        (user_hr_data['globalTime'] >= start)
        & (user_hr_data['globalTime'] <= end)
        for start, end in times
    ])
    relevant_data = user_hr_data[mask]
    total_hr_count += relevant_data['pulse'].sum()
    total_sample_count += len(relevant_data)

    try:
        avg_hr_val = total_hr_count / total_sample_count
    except:
        avg_hr_val = np.nan

    total_sleep_hr_avg[id] = avg_hr_val

  0%|          | 0/210 [00:00<?, ?it/s]

 18%|█▊        | 38/210 [03:19<14:42,  5.13s/it]C:\Users\tomda\AppData\Local\Temp\ipykernel_16236\2710335396.py:33: RuntimeWarning: invalid value encountered in scalar divide
  avg_hr_val = total_hr_count / total_sample_count
 68%|██████▊   | 142/210 [13:00<03:39,  3.23s/it]C:\Users\tomda\AppData\Local\Temp\ipykernel_16236\2710335396.py:33: RuntimeWarning: invalid value encountered in scalar divide
  avg_hr_val = total_hr_count / total_sample_count
 76%|███████▌  | 159/210 [15:27<12:04, 14.21s/it]C:\Users\tomda\AppData\Local\Temp\ipykernel_16236\2710335396.py:33: RuntimeWarning: invalid value encountered in scalar divide
  avg_hr_val = total_hr_count / total_sample_count
100%|██████████| 210/210 [20:45<00:00,  5.93s/it]


In [50]:
with open('total_sleep_hr_avg.pickle', 'wb') as f:
    pickle.dump(total_sleep_hr_avg, f)

In [41]:
for id in sleeping.keys():
    for sleep_type in sleep_types:
        if id in avg_sleep_dict:
            sleeping[id][f'avg_{sleep_type}'] = avg_sleep_dict[id][sleep_type]

In [39]:
avg_sleep_dict.pop('test', None)

{}

In [58]:
with open('sleeping.pickle', 'wb') as f:
    pickle.dump(sleeping, f)

In [55]:
with open('avg_sleeping.pickle', 'wb') as f:
    pickle.dump(avg_sleep_dict, f)

In [7]:
with open('avg_sleeping.pickle', 'rb') as f:
    avg_sleep_dict = pickle.load(f)

In [32]:
# rename the keys in sleeping data
sleep_types = ('light_sleep', 'deep_sleep', 'rem_sleep', 'awake_sleep')
for id in sleeping.keys():
    for sleep_type in sleep_types:
        old_key, new_key = f'avg_{sleep_type}', f'{sleep_type}_avg_hr'
        if id in avg_sleep_dict and old_key in sleeping[id].keys():
            sleeping[id][new_key] = sleeping[id].pop(old_key)

In [52]:
# save the total average of sleep heart rate for each id (light, deep, rem sleep)
for id in sleeping:
    if id in total_sleep_hr_avg:
        sleeping[id]['total_sleep_avg_hr'] = total_sleep_hr_avg[id]